In [97]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')

melb_df = melb_data.copy()

melb_df.drop(['index', 'Coordinates'], axis=1, inplace=True)

print(melb_data.columns)

if not all(column in melb_data.columns for column in ['Suburb']):
    print(None)


Index(['index', 'Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method',
       'SellerG', 'Date', 'Distance', 'Postcode', 'Bedroom', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount', 'Coordinates'],
      dtype='object')


In [98]:
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']

melb_df['MeanRoomsArea'] = melb_df['BuildingArea'] / total_rooms


diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
display(melb_df['AreaRatio'])

0       -0.231707
1       -0.327660
2        0.056338
3        0.145455
4        0.083969
           ...   
13575   -0.676093
13576   -0.429185
13577   -0.551601
13578   -0.693060
13579   -0.527426
Name: AreaRatio, Length: 13580, dtype: float64

In [99]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)

def get_street_type(adress):
    check_list = ['S', 'W', 'N', 'E']
    
    adress = adress.split()
    street_type = adress[-1]
    
    if street_type[-1] in check_list:
        street_type = adress[-2]
    
    return street_type
        
street_types = melb_df['Address'].apply(get_street_type)

popular_stypes = street_types.value_counts().nlargest(10).index

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else "other")

melb_df = melb_df.drop('Address', axis=1)

melb_df['StreetType']


0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [100]:
melb_df['WeekdaySale'] = melb_df['Date'].dt.weekday

def get_weekend(weekday):
    if weekday in [5, 6]:
        return 1    
    return 0

melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)

melb_df[melb_df['Weekend'] == 1]['Price'].mean()

1081198.6406956792

In [101]:
popular_sellers = melb_df['SellerG'].value_counts().nlargest(49).index

def isSelletPopular(seller):
    if seller in popular_sellers:
        return seller
    return 'other'

melb_df['SellerG'] = melb_df['SellerG'].apply(isSelletPopular)

melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min() / melb_df[melb_df['SellerG'] == 'other']['Price'].min()

1.297709923664122

In [102]:


suburb_most_popular = melb_df['Suburb'].value_counts().nlargest(119).index

def check_suburb(sub):
    if sub in suburb_most_popular:
        return sub
    return 'other'

melb_df['Suburb'] = melb_df['Suburb'].apply(check_suburb)

melb_df['Suburb'] = melb_df['Suburb'].astype('category')

melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         13580 non-null  category      
 1   Rooms          13580 non-null  int64         
 2   Type           13580 non-null  object        
 3   Price          13580 non-null  float64       
 4   Method         13580 non-null  object        
 5   SellerG        13580 non-null  object        
 6   Date           13580 non-null  datetime64[ns]
 7   Distance       13580 non-null  float64       
 8   Postcode       13580 non-null  int64         
 9   Bedroom        13580 non-null  int64         
 10  Bathroom       13580 non-null  int64         
 11  Car            13580 non-null  int64         
 12  Landsize       13580 non-null  float64       
 13  BuildingArea   13580 non-null  float64       
 14  YearBuilt      13580 non-null  int64         
 15  CouncilArea    1221